In [1]:
import numpy as np
import random as rd

In [2]:
class RandomPoint:
    """ RandomPoint class for representing and manipulating random n-dimensional coordinates. """
    
    def __init__(self, dim=1, lower=0.0, upper=1.0):
        """ Create a new random point assuming a uniform distribution between the upper and lower bounds. 
        Args:
        dim (int): Dimension.
        lower (float): Lower bound.
        upper (float): Upper bound.
        """
        self.dim = dim # dimension
        self.lower = lower # lower bound
        self.upper = upper # upper bound
        self.point = np.random.uniform(lower, upper, dim) # point (numpy.ndarray) (half open interval!)

In [3]:
class RandomLine:
    """ RandomLine class for representing and manipulating random lines in n-dimensional coordinates. """
    
    def __init__(self, dim=1, lower=0.0, upper=1.0):
        """ Create a new random line assuming a uniform distribution between the upper and lower bounds. 
        Args:
        dim (int): Dimension.
        lower (float): Lower bound.
        upper (float): Upper bound.
        """        
        self.points = [RandomPoint(dim, lower, upper) for i in range(0,2)] # inputs (list(RandomPoint))
    
    def target_function(self, randomPoint):
        """ Evaluate target function using equation of line in symmetric form.
        https://math.stackexchange.com/questions/274712/calculate-on-which-side-of-a-straight-line-is-a-given-point-located.
        https://en.wikipedia.org/wiki/Linear_equation.
        Args:
        randomPoint (RandomPoint): Random point.
        Returns: 
        int. +1 for the point being above the line. -1 for the point being below the line. 0 otherwise.     
        """
        x_minus_x1  = randomPoint.point[0] - self.points[0].point[0]
        y_minus_y1  = randomPoint.point[1] - self.points[0].point[1]
        y2_minus_y1 = self.points[1].point[1] - self.points[0].point[1]
        x2_minus_x1 = self.points[1].point[0] - self.points[0].point[0]
        
        return np.sign(x_minus_x1 * y2_minus_y1 - y_minus_y1 * x2_minus_x1)
    
    def weights(self):
        """ Evaluate weights of equation of line in symmetric form, i.e. w0 * x0 + w1 * x1 + w2 * x2 = 0
        https://math.stackexchange.com/questions/274712/calculate-on-which-side-of-a-straight-line-is-a-given-point-located.
        https://en.wikipedia.org/wiki/Linear_equation.
        Returns: 
        np.ndarray consisting of weights of the line [w0, w1, w2]    
        """
        y1_times_x2 = self.points[0].point[1] * self.points[1].point[0]
        x1_times_y2 = self.points[0].point[0] * self.points[1].point[1]
        y2_minus_y1 = self.points[1].point[1] - self.points[0].point[1]
        x2_minus_x1 = self.points[1].point[0] - self.points[0].point[0] 
        
        return np.array([y1_times_x2 - x1_times_y2, y2_minus_y1, -x2_minus_x1])
    

In [4]:
class DataSet:
    """ DataSet class for representing and manipulating random datasets. """
    def __init__(self, N=10, dim=1, lower=0.0, upper=1.0):
        """ Create a new data set assuming a uniform distribution between the 
            upper and lower bounds. 
        Args:
        N (int): Number of points.
        dim (int): Dimension of points.
        lower (float): Lower bound.
        upper (float): Upper bound.
        """
        self.inputs = [RandomPoint(dim, lower, upper) for i in range(0,N)] # inputs (list(RandomPoint))
        self.outputs = [RandomLine(2, -1.0, 1.0).target_function(input) for input in self.inputs] # outputs (list(int))
        #self.data = list(zip(self.inputs, self.outputs)) # inputs (list(Tuple(RandomPoint, int)))        

In [5]:
class PLA:
    """ Perceptron Learning Algorithm (PLA) class. """
    
    def __init__(self, initialWeights, trainingSet):
        """ Create a new Perceptron Learning Algorithm (PLA). 
        Args:
        dim (int): Dimension.
        lower (float): Lower bound.
        upper (float): Upper bound.
        """
        self.initialWeights = initialWeights # initial weights
        self.xn = np.array([np.insert(input.point, 0, 1.0) for input in trainingSet.inputs], copy=True) # training set inputs
        self.trainingSet = trainingSet
        #self.yn = np.array(trainingSet.outputs, copy=True) # training set outputs
    
    def run(self, randomLine, maxIterations=100):    
        """Run of Perceptron Learning Algorithm (PLA).
        Returns:
        g
        """        
        # calculate outputs
        #yn = np.array([randomLine.target_function(xn) for xn in self.xn], copy=True) # outputs (list(int))
        yn = np.array([randomLine.target_function(input) for input in self.trainingSet.inputs], copy=True) # outputs (list(int))
        
        # calculate target function (weights)
        targetWeights = randomLine.weights()
        #print('Target weights ', targetWeights)
        
        # misclassified points
        #misPointIdxs = self.misclassifiedPoints(self.initialWeights, yn)
          
        # PLA iterations
        numIterations = 0
        weights = np.copy(self.initialWeights) # initialize weights
        converged = False
        for i in range(0, maxIterations):
            numIterations += 1
            #print('Iteration # ', numIterations)
            
            # misclassified points
            misPointIdxs = self.misclassifiedPoints(weights, yn)
        
            # break out of loop if converged
            if len(misPointIdxs) == 0:
                converged = True
                print('PLA algorithm converged after ', numIterations, ' iterations')
                break
            
            # pick misclassified point at random
            misPointIdx = rd.choice(misPointIdxs)
            #print('Chosen misclassified point index ', misPointIdx)
            
            # update weights
            weights = np.add(weights, np.multiply(yn[misPointIdx], self.xn[misPointIdx]))
            #print('Weights ', weights)
            
        if not converged:
            raise Exception('Perceptron algorithm failed to converge after maximum number of iterations.')
        
        # estimate disagreement probability between f and g
        disagreementProb = self.disagreementProbability(1000, weights, targetWeights)
        
        return weights, numIterations, disagreementProb
    
    # private methods
    def misclassifiedPoints(self, weights, yn):
        misPointIdxs = []      
        for i in range(0, len(self.xn)):
            if np.sign(np.dot(weights, self.xn[i])) != yn[i]:
                misPointIdxs.append(i)
        #print('Misclassified point indexes', misPointIdxs)
        return misPointIdxs
    
    def disagreementProbability(self, npoints, weights, targetWeights):
        # generate random points
        estPoints = [RandomPoint(2, -1.0, 1.0) for i in range(0,npoints)]
        points = np.array([np.insert(estPoint.point, 0, 1.0) for estPoint in estPoints])
        
        # calculate probability
        disagreementProb = 0
        for point in points:
            g = np.sign(weights[0] * point[0] + weights[1] * point[1] + weights[2] * point[2])
            f = np.sign(targetWeights[0] * point[0] + targetWeights[1] * point[1] + targetWeights[2] * point[2])
            if g != f:
                disagreementProb += 1
                
        return float(disagreementProb / npoints)

In [6]:
def runPerceptron(initialWeights, trainingSet, numRuns=1, maxIterations=100):
    '''Run perceptron algorithm.
    Args: 
    numRuns (int): number of runs
    
    Returns:
    meanIterations (float): mean number of iterations for convergence
    '''
    meanIterations = 0
    meanDisagreementProb = 0.0
    for i in range(0, numRuns):
        pla = PLA(initialWeights, trainingSet)
        weights, numIterations, disagreementProb = pla.run(RandomLine(2, -1.0, 1.0), maxIterations)
        #print('Final weights ', weights)
        meanIterations += numIterations
        meanDisagreementProb += disagreementProb
    
    meanIterations = meanIterations / numRuns
    meanDisagreementProb = meanDisagreementProb / numRuns
    
    return meanIterations, meanDisagreementProb

In [7]:
trainingSet = DataSet(N=10, dim=2, lower=-1.0, upper=1.0)
meanIterations, meanDisagreementProb = runPerceptron(np.array([0.0, 0.0, 0.0]), trainingSet, numRuns=1000, maxIterations=1000)
print('Average number of iterations = ', meanIterations)
print('Average disagreement probability between f and g = ', meanDisagreementProb)

PLA algorithm converged after  34  iterations
PLA algorithm converged after  12  iterations
PLA algorithm converged after  12  iterations
PLA algorithm converged after  14  iterations
PLA algorithm converged after  5  iterations
PLA algorithm converged after  20  iterations
PLA algorithm converged after  4  iterations
PLA algorithm converged after  32  iterations
PLA algorithm converged after  20  iterations
PLA algorithm converged after  10  iterations
PLA algorithm converged after  32  iterations
PLA algorithm converged after  2  iterations
PLA algorithm converged after  10  iterations
PLA algorithm converged after  6  iterations
PLA algorithm converged after  6  iterations
PLA algorithm converged after  12  iterations
PLA algorithm converged after  10  iterations
PLA algorithm converged after  10  iterations
PLA algorithm converged after  6  iterations
PLA algorithm converged after  20  iterations
PLA algorithm converged after  5  iterations
PLA algorithm converged after  13  iterat

PLA algorithm converged after  2  iterations
PLA algorithm converged after  14  iterations
PLA algorithm converged after  42  iterations
PLA algorithm converged after  22  iterations
PLA algorithm converged after  78  iterations
PLA algorithm converged after  3  iterations
PLA algorithm converged after  4  iterations
PLA algorithm converged after  8  iterations
PLA algorithm converged after  15  iterations
PLA algorithm converged after  5  iterations
PLA algorithm converged after  2  iterations
PLA algorithm converged after  12  iterations
PLA algorithm converged after  2  iterations
PLA algorithm converged after  5  iterations
PLA algorithm converged after  10  iterations
PLA algorithm converged after  3  iterations
PLA algorithm converged after  2  iterations
PLA algorithm converged after  5  iterations
PLA algorithm converged after  8  iterations
PLA algorithm converged after  12  iterations
PLA algorithm converged after  3  iterations
PLA algorithm converged after  14  iterations
P

PLA algorithm converged after  11  iterations
PLA algorithm converged after  40  iterations
PLA algorithm converged after  18  iterations
PLA algorithm converged after  10  iterations
PLA algorithm converged after  25  iterations
PLA algorithm converged after  3  iterations
PLA algorithm converged after  4  iterations
PLA algorithm converged after  2  iterations
PLA algorithm converged after  7  iterations
PLA algorithm converged after  26  iterations
PLA algorithm converged after  13  iterations
PLA algorithm converged after  5  iterations
PLA algorithm converged after  5  iterations
PLA algorithm converged after  4  iterations
PLA algorithm converged after  16  iterations
PLA algorithm converged after  9  iterations
PLA algorithm converged after  4  iterations
PLA algorithm converged after  18  iterations
PLA algorithm converged after  3  iterations
PLA algorithm converged after  6  iterations
PLA algorithm converged after  10  iterations
PLA algorithm converged after  12  iterations

PLA algorithm converged after  20  iterations
PLA algorithm converged after  3  iterations
PLA algorithm converged after  6  iterations
PLA algorithm converged after  3  iterations
PLA algorithm converged after  14  iterations
PLA algorithm converged after  18  iterations
PLA algorithm converged after  88  iterations
PLA algorithm converged after  2  iterations
PLA algorithm converged after  5  iterations
PLA algorithm converged after  6  iterations
PLA algorithm converged after  24  iterations
PLA algorithm converged after  5  iterations
PLA algorithm converged after  6  iterations
PLA algorithm converged after  3  iterations
PLA algorithm converged after  5  iterations
PLA algorithm converged after  12  iterations
PLA algorithm converged after  5  iterations
PLA algorithm converged after  10  iterations
PLA algorithm converged after  3  iterations
PLA algorithm converged after  2  iterations
PLA algorithm converged after  10  iterations
PLA algorithm converged after  3  iterations
PL

PLA algorithm converged after  38  iterations
PLA algorithm converged after  3  iterations
PLA algorithm converged after  6  iterations
PLA algorithm converged after  16  iterations
PLA algorithm converged after  8  iterations
PLA algorithm converged after  13  iterations
PLA algorithm converged after  6  iterations
PLA algorithm converged after  6  iterations
PLA algorithm converged after  6  iterations
PLA algorithm converged after  47  iterations
PLA algorithm converged after  14  iterations
PLA algorithm converged after  6  iterations
PLA algorithm converged after  6  iterations
PLA algorithm converged after  11  iterations
PLA algorithm converged after  21  iterations
PLA algorithm converged after  11  iterations
PLA algorithm converged after  52  iterations
PLA algorithm converged after  7  iterations
PLA algorithm converged after  2  iterations
PLA algorithm converged after  27  iterations
PLA algorithm converged after  2  iterations
PLA algorithm converged after  5  iterations


PLA algorithm converged after  2  iterations
PLA algorithm converged after  16  iterations
PLA algorithm converged after  7  iterations
PLA algorithm converged after  6  iterations
PLA algorithm converged after  6  iterations
PLA algorithm converged after  2  iterations
PLA algorithm converged after  6  iterations
PLA algorithm converged after  11  iterations
PLA algorithm converged after  10  iterations
PLA algorithm converged after  3  iterations
PLA algorithm converged after  6  iterations
PLA algorithm converged after  3  iterations
PLA algorithm converged after  5  iterations
PLA algorithm converged after  22  iterations
PLA algorithm converged after  27  iterations
PLA algorithm converged after  7  iterations
PLA algorithm converged after  36  iterations
PLA algorithm converged after  3  iterations
PLA algorithm converged after  2  iterations
PLA algorithm converged after  4  iterations
PLA algorithm converged after  12  iterations
PLA algorithm converged after  17  iterations
PL

In [8]:
trainingSet = DataSet(N=100, dim=2, lower=-1.0, upper=1.0)
meanIterations, meanDisagreementProb = runPerceptron(np.array([0.0, 0.0, 0.0]), trainingSet, numRuns=1000, maxIterations=100000)
print('Average number of iterations = ', meanIterations)
print('Average disagreement probability between f and g = ', meanDisagreementProb)

PLA algorithm converged after  68  iterations
PLA algorithm converged after  25  iterations
PLA algorithm converged after  212  iterations
PLA algorithm converged after  16  iterations
PLA algorithm converged after  30  iterations
PLA algorithm converged after  33  iterations
PLA algorithm converged after  57  iterations
PLA algorithm converged after  30  iterations
PLA algorithm converged after  104  iterations
PLA algorithm converged after  34  iterations
PLA algorithm converged after  232  iterations
PLA algorithm converged after  34  iterations
PLA algorithm converged after  13  iterations
PLA algorithm converged after  32  iterations
PLA algorithm converged after  178  iterations
PLA algorithm converged after  59  iterations
PLA algorithm converged after  6  iterations
PLA algorithm converged after  145  iterations
PLA algorithm converged after  32  iterations
PLA algorithm converged after  1196  iterations
PLA algorithm converged after  40  iterations
PLA algorithm converged afte

PLA algorithm converged after  5  iterations
PLA algorithm converged after  283  iterations
PLA algorithm converged after  390  iterations
PLA algorithm converged after  36  iterations
PLA algorithm converged after  132  iterations
PLA algorithm converged after  56  iterations
PLA algorithm converged after  74  iterations
PLA algorithm converged after  28  iterations
PLA algorithm converged after  32  iterations
PLA algorithm converged after  94  iterations
PLA algorithm converged after  11  iterations
PLA algorithm converged after  67  iterations
PLA algorithm converged after  38  iterations
PLA algorithm converged after  161  iterations
PLA algorithm converged after  32  iterations
PLA algorithm converged after  29  iterations
PLA algorithm converged after  18  iterations
PLA algorithm converged after  74  iterations
PLA algorithm converged after  110  iterations
PLA algorithm converged after  101  iterations
PLA algorithm converged after  662  iterations
PLA algorithm converged afte

PLA algorithm converged after  68  iterations
PLA algorithm converged after  61  iterations
PLA algorithm converged after  78  iterations
PLA algorithm converged after  1224  iterations
PLA algorithm converged after  73  iterations
PLA algorithm converged after  21  iterations
PLA algorithm converged after  59  iterations
PLA algorithm converged after  181  iterations
PLA algorithm converged after  64  iterations
PLA algorithm converged after  82  iterations
PLA algorithm converged after  50  iterations
PLA algorithm converged after  50  iterations
PLA algorithm converged after  54  iterations
PLA algorithm converged after  152  iterations
PLA algorithm converged after  868  iterations
PLA algorithm converged after  94  iterations
PLA algorithm converged after  34  iterations
PLA algorithm converged after  27  iterations
PLA algorithm converged after  57  iterations
PLA algorithm converged after  568  iterations
PLA algorithm converged after  22  iterations
PLA algorithm converged afte

PLA algorithm converged after  136  iterations
PLA algorithm converged after  159  iterations
PLA algorithm converged after  47  iterations
PLA algorithm converged after  32  iterations
PLA algorithm converged after  74  iterations
PLA algorithm converged after  314  iterations
PLA algorithm converged after  458  iterations
PLA algorithm converged after  46  iterations
PLA algorithm converged after  25  iterations
PLA algorithm converged after  37  iterations
PLA algorithm converged after  55  iterations
PLA algorithm converged after  61  iterations
PLA algorithm converged after  16  iterations
PLA algorithm converged after  284  iterations
PLA algorithm converged after  71  iterations
PLA algorithm converged after  116  iterations
PLA algorithm converged after  150  iterations
PLA algorithm converged after  46  iterations
PLA algorithm converged after  50  iterations
PLA algorithm converged after  46  iterations
PLA algorithm converged after  54  iterations
PLA algorithm converged aft

PLA algorithm converged after  248  iterations
PLA algorithm converged after  13  iterations
PLA algorithm converged after  36  iterations
PLA algorithm converged after  28  iterations
PLA algorithm converged after  41  iterations
PLA algorithm converged after  117  iterations
PLA algorithm converged after  35  iterations
PLA algorithm converged after  53  iterations
PLA algorithm converged after  230  iterations
PLA algorithm converged after  76  iterations
PLA algorithm converged after  90  iterations
PLA algorithm converged after  64  iterations
PLA algorithm converged after  35  iterations
PLA algorithm converged after  206  iterations
PLA algorithm converged after  85  iterations
PLA algorithm converged after  53  iterations
PLA algorithm converged after  15  iterations
PLA algorithm converged after  360  iterations
PLA algorithm converged after  167  iterations
PLA algorithm converged after  54  iterations
PLA algorithm converged after  49  iterations
PLA algorithm converged afte

PLA algorithm converged after  30  iterations
PLA algorithm converged after  29  iterations
PLA algorithm converged after  93  iterations
PLA algorithm converged after  66  iterations
PLA algorithm converged after  63  iterations
PLA algorithm converged after  45  iterations
PLA algorithm converged after  55  iterations
PLA algorithm converged after  414  iterations
PLA algorithm converged after  135  iterations
PLA algorithm converged after  18  iterations
PLA algorithm converged after  58  iterations
PLA algorithm converged after  38  iterations
PLA algorithm converged after  46  iterations
PLA algorithm converged after  128  iterations
PLA algorithm converged after  39  iterations
PLA algorithm converged after  3  iterations
PLA algorithm converged after  274  iterations
PLA algorithm converged after  132  iterations
PLA algorithm converged after  117  iterations
PLA algorithm converged after  39  iterations
PLA algorithm converged after  38  iterations
PLA algorithm converged after

In [9]:
trainingSet = DataSet(N=1000, dim=2, lower=-1.0, upper=1.0)
meanIterations, meanDisagreementProb = runPerceptron(np.array([0.0, 0.0, 0.0]), trainingSet, numRuns=1000, maxIterations=1000000)
print('Average number of iterations = ', meanIterations)
print('Average disagreement probability between f and g = ', meanDisagreementProb)

PLA algorithm converged after  319  iterations
PLA algorithm converged after  493  iterations
PLA algorithm converged after  399  iterations
PLA algorithm converged after  373  iterations
PLA algorithm converged after  610  iterations
PLA algorithm converged after  396  iterations
PLA algorithm converged after  345  iterations
PLA algorithm converged after  843  iterations
PLA algorithm converged after  1387  iterations
PLA algorithm converged after  935  iterations
PLA algorithm converged after  498  iterations
PLA algorithm converged after  741  iterations
PLA algorithm converged after  871  iterations
PLA algorithm converged after  746  iterations
PLA algorithm converged after  491  iterations
PLA algorithm converged after  382  iterations
PLA algorithm converged after  2024  iterations
PLA algorithm converged after  1236  iterations
PLA algorithm converged after  182  iterations
PLA algorithm converged after  90  iterations
PLA algorithm converged after  266  iterations
PLA algorit

PLA algorithm converged after  332  iterations
PLA algorithm converged after  137  iterations
PLA algorithm converged after  2144  iterations
PLA algorithm converged after  479  iterations
PLA algorithm converged after  739  iterations
PLA algorithm converged after  444  iterations
PLA algorithm converged after  417  iterations
PLA algorithm converged after  289  iterations
PLA algorithm converged after  579  iterations
PLA algorithm converged after  487  iterations
PLA algorithm converged after  228  iterations
PLA algorithm converged after  1413  iterations
PLA algorithm converged after  394  iterations
PLA algorithm converged after  536  iterations
PLA algorithm converged after  399  iterations
PLA algorithm converged after  1697  iterations
PLA algorithm converged after  802  iterations
PLA algorithm converged after  153  iterations
PLA algorithm converged after  601  iterations
PLA algorithm converged after  13868  iterations
PLA algorithm converged after  423  iterations
PLA algo

PLA algorithm converged after  2286  iterations
PLA algorithm converged after  1369  iterations
PLA algorithm converged after  173  iterations
PLA algorithm converged after  172  iterations
PLA algorithm converged after  449  iterations
PLA algorithm converged after  817  iterations
PLA algorithm converged after  492  iterations
PLA algorithm converged after  471  iterations
PLA algorithm converged after  277  iterations
PLA algorithm converged after  517  iterations
PLA algorithm converged after  423  iterations
PLA algorithm converged after  361  iterations
PLA algorithm converged after  828  iterations
PLA algorithm converged after  258  iterations
PLA algorithm converged after  171  iterations
PLA algorithm converged after  2071  iterations
PLA algorithm converged after  666  iterations
PLA algorithm converged after  2364  iterations
PLA algorithm converged after  524  iterations
PLA algorithm converged after  536  iterations
PLA algorithm converged after  223  iterations
PLA algor

PLA algorithm converged after  256  iterations
PLA algorithm converged after  781  iterations
PLA algorithm converged after  303  iterations
PLA algorithm converged after  3348  iterations
PLA algorithm converged after  658  iterations
PLA algorithm converged after  397  iterations
PLA algorithm converged after  155  iterations
PLA algorithm converged after  273  iterations
PLA algorithm converged after  284  iterations
PLA algorithm converged after  1372  iterations
PLA algorithm converged after  236  iterations
PLA algorithm converged after  412  iterations
PLA algorithm converged after  399  iterations
PLA algorithm converged after  654  iterations
PLA algorithm converged after  68  iterations
PLA algorithm converged after  1852  iterations
PLA algorithm converged after  799  iterations
PLA algorithm converged after  616  iterations
PLA algorithm converged after  869  iterations
PLA algorithm converged after  618  iterations
PLA algorithm converged after  216  iterations
PLA algorit

PLA algorithm converged after  539  iterations
PLA algorithm converged after  225  iterations
PLA algorithm converged after  311  iterations
PLA algorithm converged after  437  iterations
PLA algorithm converged after  744  iterations
PLA algorithm converged after  1611  iterations
PLA algorithm converged after  316  iterations
PLA algorithm converged after  580  iterations
PLA algorithm converged after  571  iterations
PLA algorithm converged after  170  iterations
PLA algorithm converged after  980  iterations
PLA algorithm converged after  279  iterations
PLA algorithm converged after  472  iterations
PLA algorithm converged after  3382  iterations
PLA algorithm converged after  176  iterations
PLA algorithm converged after  436  iterations
PLA algorithm converged after  1861  iterations
PLA algorithm converged after  552  iterations
PLA algorithm converged after  3285  iterations
PLA algorithm converged after  297  iterations
PLA algorithm converged after  2178  iterations
PLA algo

PLA algorithm converged after  302  iterations
PLA algorithm converged after  785  iterations
PLA algorithm converged after  760  iterations
PLA algorithm converged after  1044  iterations
PLA algorithm converged after  1535  iterations
PLA algorithm converged after  296  iterations
PLA algorithm converged after  1515  iterations
PLA algorithm converged after  452  iterations
PLA algorithm converged after  6032  iterations
PLA algorithm converged after  546  iterations
PLA algorithm converged after  837  iterations
PLA algorithm converged after  500  iterations
PLA algorithm converged after  906  iterations
PLA algorithm converged after  1338  iterations
PLA algorithm converged after  442  iterations
PLA algorithm converged after  491  iterations
PLA algorithm converged after  1679  iterations
PLA algorithm converged after  1665  iterations
PLA algorithm converged after  243  iterations
PLA algorithm converged after  271  iterations
PLA algorithm converged after  164  iterations
PLA al